In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import re
%matplotlib inline

In [2]:
master = pd.read_csv('data/Master.csv', parse_dates=['debut', 'finalGame'])
# Currently not caring about pitchers

In [3]:
master.dtypes

playerID                object
birthYear              float64
birthMonth             float64
birthDay               float64
birthCountry            object
birthState              object
birthCity               object
deathYear              float64
deathMonth             float64
deathDay               float64
deathCountry            object
deathState              object
deathCity               object
nameFirst               object
nameLast                object
nameGiven               object
weight                 float64
height                 float64
bats                    object
throws                  object
debut           datetime64[ns]
finalGame       datetime64[ns]
retroID                 object
bbrefID                 object
dtype: object

In [4]:
def birth_state_count(state):
    print(master[master['birthState'] == state].size)

birth_state_count('RI')
birth_state_count('FL')

1848
11256


In [5]:
bbmaster = master[['playerID', 'nameFirst', 'nameLast', 'nameGiven']]
bbmaster.head()

,playerID,nameFirst,nameLast,nameGiven
0,aardsda01,David,Aardsma,David Allan
1,aaronha01,Hank,Aaron,Henry Louis
2,aaronto01,Tommie,Aaron,Tommie Lee
3,aasedo01,Don,Aase,Donald William
4,abadan01,Andy,Abad,Fausto Andres


In [6]:
batting = pd.read_csv('data/Batting.csv')
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [7]:
batting.head()

,playerID,yearID,stint,teamID,lgID,G,AB,R,H,2B,...,RBI,SB,CS,BB,SO,IBB,HBP,SH,SF,GIDP
0,abercda01,1871,1,TRO,NaN,1,4,0,0,0,...,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN
1,addybo01,1871,1,RC1,NaN,25,118,30,32,6,...,13,8,1,4,0,NaN,NaN,NaN,NaN,NaN
2,allisar01,1871,1,CL1,NaN,29,137,28,40,4,...,19,3,1,2,5,NaN,NaN,NaN,NaN,NaN
3,allisdo01,1871,1,WS3,NaN,27,133,28,44,10,...,27,1,1,0,2,NaN,NaN,NaN,NaN,NaN
4,ansonca01,1871,1,RC1,NaN,25,120,29,39,11,...,16,6,2,2,1,NaN,NaN,NaN,NaN,NaN


In [8]:
master_batting = pd.merge(master, batting)

In [9]:
batting.columns

Index(['playerID', 'yearID', 'stint', 'teamID', 'lgID', 'G', 'AB', 'R', 'H',
       '2B', '3B', 'HR', 'RBI', 'SB', 'CS', 'BB', 'SO', 'IBB', 'HBP', 'SH',
       'SF', 'GIDP'],
      dtype='object')

In [10]:
bats = batting[['playerID', 'yearID','AB', 'H', 'BB', 'HBP', 'SF']]
bats.head()
print(len(bats))
bbdata2 = pd.merge(bbmaster, bats)
bbdata2.head()

99846


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0
1,aardsda01,David,Aardsma,David Allan,2006,2,0,0,0,0
2,aardsda01,David,Aardsma,David Allan,2007,0,0,0,0,0
3,aardsda01,David,Aardsma,David Allan,2008,1,0,0,0,0
4,aardsda01,David,Aardsma,David Allan,2009,0,0,0,0,0


In [11]:
salaries = pd.read_csv('data/Salaries.csv')
salaries.columns

Index(['yearID', 'teamID', 'lgID', 'playerID', 'salary'], dtype='object')

In [12]:
salaries = salaries[['yearID', 'playerID', 'salary']]
print(len(salaries))
salaries.head()

24758


,yearID,playerID,salary
0,1985,barkele01,870000
1,1985,bedrost01,550000
2,1985,benedbr01,545000
3,1985,campri01,633333
4,1985,ceronri01,625000


In [13]:
bbdata_sal = pd.merge(bbdata2, salaries)
print(len(bbdata_sal))
bbdata_sal.head()

26470


,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary
0,aardsda01,David,Aardsma,David Allan,2004,0,0,0,0,0,300000
1,aardsda01,David,Aardsma,David Allan,2007,0,0,0,0,0,387500
2,aardsda01,David,Aardsma,David Allan,2008,1,0,0,0,0,403250
3,aardsda01,David,Aardsma,David Allan,2009,0,0,0,0,0,419000
4,aardsda01,David,Aardsma,David Allan,2010,0,0,0,0,0,2750000


In [14]:
# Drop all AB values of 0 or null
bbdata_sal = bbdata_sal.drop(bbdata_sal.index[(bbdata_sal['AB'] < 15) | (pd.isnull(bbdata_sal['AB']))])

In [15]:
bbdata_sal['OBP'] = bbdata_sal.apply(lambda row: (row['H'] + row['BB'] + row['HBP']) * 100 / \
                                     (row['AB'] + row['BB'] + row['HBP'] + row['SF']), axis=1)

In [16]:
bbdata_sal.head()

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
14,abbotje01,Jeff,Abbott,Jeffrey William,1998,244,68,9,0,5,175000,29.844961
15,abbotje01,Jeff,Abbott,Jeffrey William,1999,57,9,5,0,1,255000,22.222222
16,abbotje01,Jeff,Abbott,Jeffrey William,2000,215,59,21,2,1,255000,34.309623
17,abbotje01,Jeff,Abbott,Jeffrey William,2001,42,11,3,1,0,300000,32.608696
27,abbotji01,Jim,Abbott,James Anthony,1999,21,2,0,0,0,400000,9.523810


In [17]:
bbdata_sal.sort(columns=['OBP'], ascending=False).head()

,playerID,nameFirst,nameLast,nameGiven,yearID,AB,H,BB,HBP,SF,salary,OBP
2367,bondsba01,Barry,Bonds,Barry Lamar,2004,373,135,232,9,3,18000000,60.940032
2365,bondsba01,Barry,Bonds,Barry Lamar,2002,403,149,198,9,2,15000000,58.169935
15947,millira01,Randy,Milligan,Randy Andre,1993,47,20,14,0,0,635000,55.737705
8255,gibbojo02,John,Gibbons,John Michael,1986,19,9,3,0,0,65000,54.545455
21377,schatda01,Dan,Schatzeder,Daniel Ernest,1986,21,9,5,0,0,415000,53.846154


In [18]:
bbdata_sal.drop(bbdata_sal.index[bbdata_sal['salary'] <= 0])
pass

In [19]:
print(bbdata_sal.iloc[11611]['OBP'] / bbdata_sal.iloc[11611]['salary'])
print(bbdata_sal.iloc[2367]['OBP'] / bbdata_sal.iloc[2367]['salary'])
bbdata_sal.iloc[11611]

6.33562286529e-06
2.28848821082e-05


playerID        phillbr01
nameFirst         Brandon
nameLast         Phillips
nameGiven    Brandon Emil
yearID               2009
AB                    584
H                     161
BB                     44
HBP                     6
SF                      8
salary            5187500
OBP              32.86604
Name: 18776, dtype: object

In [20]:
bbdata_sal['OBP_salary_ratio'] = bbdata_sal.apply(lambda row: row['OBP'] / row['salary'], axis=1)

ZeroDivisionError: ('float division by zero', 'occurred at index 11611')

In [21]:
bbdata_sal.iloc[2367]['salary']

1400000

In [ ]:
appearances = pd.read_csv('data/Appearances.csv')
appearances.columns

In [ ]:
appearances.pop('lgID')
appearances.pop('teamID')
appearances.head()

In [ ]:
player_info = pd.merge(bbdata_sal, appearances)
player_info.head()

In [ ]:
# master_batting_appearances = pd.merge(master_batting, appearances)

In [ ]:
# bbdata = pd.merge(master_batting_appearances, salaries)

In [ ]:
# bbdata.head()